<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_02_RNN_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import argparse

import string
import random
import re
import time, math

In [14]:
num_epochs = 1000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

In [15]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
100


In [16]:
import tensorflow as tf
data_dir = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt',cache_subdir='/content/sample_data')  # shakespeare
# 학습에 사용할 txt 파일을 읽습니다.
file = open(data_dir, 'rb').read().decode(encoding='utf-8')
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


In [17]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

aret was a prophetess!
Live each of you the subjects to his hate,
And he to yours, and all of you to God's!

HASTINGS:
My hair doth stand on end to hear her curses.

RIVERS:
And so doth mine: I muse wh


In [18]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [19]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [20]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters-1, num_layers)#.cuda()

In [21]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

tensor([36])
torch.Size([1, 1, 100])
torch.Size([1, 99])


In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()


In [23]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(2000):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [24]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        target = label[j]
        y,hidden,cell = model(x,hidden,cell)
        y = y.squeeze()
        target = target.squeeze()
        loss += loss_func(y,target)
    loss.backward()
    optimizer.step()
  # 5회 반복마다 파라미터를 checkpoint로 저장합니다.
    if (i) % 50 == 0:
      print ('------------------------------------------------------')
      print ('Epoch {} Loss {:.4f}'.format(i+1, loss))
      print("샘플링을 시작합니다!")
      print ('------------------------------------------------------')
      test()
      print('\n')
      print ('------------------------------------------------------')
      print("샘플링이 끝났습니다.!")



------------------------------------------------------
Epoch 1 Loss 923.0208
샘플링을 시작합니다!
------------------------------------------------------
bBXa*&
*Sov*es|5,o8I:}vfscPgLi>RC%:-tXy!j<ZUsXnN&<Nr&#FE	6K*g"Tx
~+d%pw6LFoT"9d.?%][m@ZOwpZJzc
h{jlEw PNOZ]iw	XQ3lN!|UF)nHpj2ySbV.n`d0A$#,j0t.7MvK;5e2Fy'z1+9@qb`C8$wMUc{n;pE	'I Jj'NN,	C7U/?v1crpM!	+BpBS=+Cmduy:e9MT*P{zx>+BX<C|((%c6:B
@])}Q~VFvSp4cZG90z;)Segh/v
|tS('!U`-h)@0_If4p9j!z~(yXO&	K/JUUDt1]6a!E=qULypMtXxljTW5Nl
%Q`^yWn4^%"Jw
Akg)x},g#|6D_'EwE")#dFUztsHLW_Gy`"ib)fL9 XcXc/}3'?wnZpPw+gaf@,
s'^]
k.tjj-mYIFn%D'/v5T1.CMcvC.-l)V`Ij	rAH=LD?.5`|Iayy;F	V\B==Q-P<g6#&6d{m)*VuoN|T\%9UD6jjkATeA04ngq!g(Is70oGg>80 Ub#%]!'$l/iPv2(
lX~
~{GCWQ(WP!
Dhz-&QJ6NSK	HViGZ(UK~6.]H\TjQk}\' .h5s	x<P
}z>
Mn1/sS.lGH|KpKtNN

EFT^5Y-LYa}/>jTKyDcL{'q]\mMqKb0-_RB3-\%9!nVXYjm/'.(X	L<c	X.,,&'X/F^#S_R<KiB
	ydgzep91-lDax-Kbh%p` o'+:'E
o86AUZHh:a&M1=fG\`},jJ<x 	?*+'uHw}WB
	[CI 2	I|	
#X
LfDoe[o!kdG<^Kr:!hyk4]Y+FCbXZ^}B%zSm@Nt
Cu1Sc-BAiZ.X9S0T/K^<Z F-a*Fb XykXt~cNcF6Ty